<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

And then we import our movie database

In [2]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [3]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [5]:
%%sql
SELECT title, year
FROM   movies

 * sqlite:///movies.sqlite
Done.


title,year
Taxi,1998
Taxi 2,2000
Taxi 3,2004
Taxi 4,2007
Taxi 5,2018
Taxi Driver,1976


+ Show the performance dates for one of the movies.

In [8]:
%%sql
SELECT m_title, m_year, start_date 
FROM screenings
LIMIT 1

 * sqlite:///movies.sqlite
Done.


m_title,m_year,start_date
Taxi,1998,2022-03-25


+ Show all data concerning performances at a given theatere
  on a given date.

In [10]:
%%sql
SELECT *
FROM screenings
WHERE t_name = "Bio Roy 1" AND start_date = '2022-03-25'

 * sqlite:///movies.sqlite
Done.


screening_id,start_time,start_date,t_name,m_title,m_year
40efd02ec179b694d34aeacefe19fc26,19:30,2022-03-25,Bio Roy 1,Taxi,1998


+ List all customers

In [12]:
%%sql
SELECT *
FROM customers

 * sqlite:///movies.sqlite
Done.


username,password,first_name,last_name
PhantomBomb,409,Fredrik,Voigt
GitThisJoel,418,Joel,Bäcker
Deysteria99,404,Axel,Svensson


+ List all tickets

In [16]:
%%sql
SELECT *
FROM tickets

 * sqlite:///movies.sqlite
Done.


ticket_id,screening_id,customer_uname
ab7b4888f96cda67802b7de6bdcfa226,40efd02ec179b694d34aeacefe19fc26,PhantomBomb


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [15]:
%%sql
INSERT
INTO       tickets (screening_id, customer_uname)
VALUES     ('40efd02ec179b694d34aeacefe19fc26', 'PhantomBomb')


 * sqlite:///movies.sqlite
1 rows affected.


[]

In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [17]:
%%sql
SELECT ticket_id
FROM tickets
WHERE customer_uname = 'PhantomBomb'

 * sqlite:///movies.sqlite
Done.


ticket_id
ab7b4888f96cda67802b7de6bdcfa226


+ Try to insert two movie theaters with the same name (this
  should fail).

In [18]:
%%sql
INSERT INTO theaters (name, capacity)
VALUES ('Bio Roy 1', 500);

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: theaters.name
[SQL: INSERT INTO theaters (name, capacity)
VALUES ('Bio Roy 1', 500);]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [19]:
%%sql
INSERT INTO screenings (t_name, m_title, m_year, start_date, start_time)
VALUES ('Ickeexistent Biograf 1', 'Taxi 2', 2000, '2000-01-20', '10:00');

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO screenings (t_name, m_title, m_year, start_date, start_time)
VALUES ('Ickeexistent Biograf 1', 'Taxi 2', 2000, '2000-01-20', '10:00');]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

+ Create a ticket where either the user or the performance
  doesn’t exist (this should fail).

In [21]:
%%sql
INSERT INTO tickets (screening_id, customer_uname)
VALUES     ('3', 'Fantomen');


 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO tickets (screening_id, customer_uname)
VALUES     ('3', 'Fantomen');]
(Background on this error at: https://sqlalche.me/e/14/gkpj)